# Starters
## Import Packages

In [1]:
from urllib import request
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np

In [2]:
import scipy.stats as st
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [3]:
sns.set_style('darkgrid')

## Helpers functions

In [4]:
# Function to quickly describe data
def data_desc(my_imported_data):
    print(my_imported_data.dropna(how = 'all').shape) #pour voir si certaines lignes comportent des valeurs NaN partout (Non)
    print(my_imported_data.shape)
    return(my_imported_data.dtypes)
# -------------------------------------------------------------------------------
# Function to return in a dictionary ratio of nan values per variable sorted in descending order
def display_nan_Values(my_imported_data):
    nan_dict = {my_imported_data.columns[var]: my_imported_data.iloc[:,var].isna().mean()
                     for var in range(my_imported_data.shape[1])}
    sorted_dict_by_nanvalues = sorted(nan_dict.items(), key=lambda x:x[1], reverse=True)
    sorted_dict = dict(sorted_dict_by_nanvalues)
    return sorted_dict

# -------------------------------------------------------------------------------
# Function to estimate percentage of null values for each variable in the dataset
def compute_nan_P(dataset):
    data_isna_P = (dataset.isna().sum()) * 100/len(dataset)
    cols = [x for x in range (dataset.shape[1])]
    count =0
    for ii in data_isna_P.index:
        count +=1
        if (data_isna_P[ii] > 55):
            #countna = +1
            cols.remove(count-1)
            print (ii)
        else:
            continue
    return (data_isna_P, cols)
# -------------------------------------------------------------------------------

# Function for replacing UPPERCASE with lowercase in variables names using title()
def columns_title_format(my_imported_data): 
    data_columns = list(my_imported_data.columns)
    data_columns = [col.title() for col in data_columns]
    print(data_columns)
    my_imported_data.columns = data_columns
    my_imported_data.head()
    return my_imported_data

# -------------------------------------------------------------------------------
# Function for replacing space with '_' in columns of a df
def replace_space(dataset):
    df = dataset.select_dtypes(include=['object'])
    df = df.stack().str.replace(' ', '_').unstack()
    #dataset.loc[:, df.columns] = df[:]
    # modif du 8/2/2023
    #dataset.B = df1.E
    for col in df.columns:
        dataset[col] = df[col]
    
    #del df
    return dataset

# -------------------------------------------------------------------------------
# Function To project a variable onto a dataframe
def present_variable_in_tab(my_dataset, variable_col, variable_new_name):
    effectifs = my_dataset[variable_col].value_counts()
    modalites = effectifs.index # l'index des effectifs contient les modalités

    tab = pd.DataFrame(modalites, columns = [variable_new_name]) # création du tableau à partir des modalités
    tab["n"] = effectifs.values
    tab["f"] = tab["n"] / len(my_dataset) # len(data) renvoie la taille de l'échantillon
    tab = tab.sort_values(by=[variable_new_name]) #n would be of use if many values had a on the first column of nutrigrade
    #tab.index = [1, 2, 3 ,4 ,5]
    return(tab)
# -------------------------------------------------------------------------------

In [5]:
# Function to drop rows/products with a specified variable's values (nutrigrade=NaN)
def droping_nan_variable(variable_col, my_dataset):
    # variable_Col is to be given between quotations
    cond2_droping_nan_Variable = my_dataset[variable_col].notna()
    df_new_data = my_dataset[cond2_droping_nan_Variable]
    return df_new_data
# -------------------------------------------------------------------------------

In [6]:
def replace_space(dataset):
    df = dataset.select_dtypes(include=['object'])
    df = df.stack().str.replace(' ', '_').unstack()
    dataset.loc[:, df.columns] = df[:]
    del df
    return dataset

In [7]:
# Function to compute linear regression
def iterate_regression(X, y):
    Lr = LinearRegression()

    # step1(): train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8)        
    plt.scatter(x_train, y_train, label = 'Training data', color = 'red', alpha=.7)
    plt.scatter (x_test, y_test, label ='Testing data', color ='g', alpha=.7)
    plt.legend()
    plt.title('Test Train Split')
    plt.show()
    # step2(): fit
    Lr.fit(x_train.values.reshape(-1, 1), y_train.values)
    # step3(): predict
    prediction = Lr.predict(x_test.values.reshape(-1, 1))
    plt.plot(x_test, prediction, label ='Linear Regression', color ='b')
    plt.scatter(x_test, y_test, label ='Test data', color ='g', alpha=.7)
    plt.legend()
    plt.show()
    # step 4: Compute score 
    FirstReg_score = Lr.score(x_test.values.reshape(-1, 1), y_test.values)
    Lr.get_params(deep=True)

In [8]:
def correlation_graph(pca, 
                      x_y, 
                      features) : 
    """Affiche le graphe des correlations

    Positional arguments : 
    -----------------------------------
    pca : sklearn.decomposition.PCA : notre objet PCA qui a été fit
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features : list ou tuple : la liste des features (ie des dimensions) à représenter
    """

    # Extrait x et y 
    x,y=x_y

    # Taille de l'image (en inches)
    fig, ax = plt.subplots(figsize=(10, 9))

    # Pour chaque composante : 
    for i in range(0, pca.components_.shape[1]):

        # Les flèches
        ax.arrow(0,0, 
                pca.components_[x, i],  
                pca.components_[y, i],  
                head_width=0.07,
                head_length=0.07, 
                width=0.02, )

        # Les labels
        plt.text(pca.components_[x, i] + 0.05,
                pca.components_[y, i] + 0.05,
                features[i])
        
    # Affichage des lignes horizontales et verticales
    plt.plot([-1, 1], [0, 0], color='grey', ls='--')
    plt.plot([0, 0], [-1, 1], color='grey', ls='--')

    # Nom des axes, avec le pourcentage d'inertie expliqué
    plt.xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    plt.ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))

    # Le cercle 
    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale

    # Axes et display
    plt.axis('equal')
    plt.show(block=False)


In [9]:
# Function to display fcatorial planes
def display_factorial_planes(   X_projected, 
                                x_y, 
                                pca=None, 
                                labels = None,
                                clusters=None, 
                                alpha=1,
                                figsize=[10,8], 
                                marker="." ):
    """
    Affiche la projection des individus

    Positional arguments : 
    -------------------------------------
    X_projected : np.array, pd.DataFrame, list of list : la matrice des points projetés
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2

    Optional arguments : 
    -------------------------------------
    pca : sklearn.decomposition.PCA : un objet PCA qui a été fit, cela nous permettra d'afficher la variance de chaque composante, default = None
    labels : list ou tuple : les labels des individus à projeter, default = None
    clusters : list ou tuple : la liste des clusters auquel appartient chaque individu, default = None
    alpha : float in [0,1] : paramètre de transparence, 0=100% transparent, 1=0% transparent, default = 1
    figsize : list ou tuple : couple width, height qui définit la taille de la figure en inches, default = [10,8] 
    marker : str : le type de marker utilisé pour représenter les individus, points croix etc etc, default = "."
    """

    # Transforme X_projected en np.array
    X_ = np.array(X_projected)

    # On définit la forme de la figure si elle n'a pas été donnée
    if not figsize: 
        figsize = (7,6)

    # On gère les labels
    if  labels is None : 
        labels = []
    try : 
        len(labels)
    except Exception as e : 
        raise e

    # On vérifie la variable axis 
    if not len(x_y) ==2 : 
        raise AttributeError("2 axes sont demandées")   
    if max(x_y )>= X_.shape[1] : 
        raise AttributeError("la variable axis n'est pas bonne")   

    # on définit x et y 
    x, y = x_y

    # Initialisation de la figure       
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # On vérifie s'il y a des clusters ou non
    c = None if clusters is None else clusters
 
    sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y], hue=c)

    # Si la variable pca a été fournie, on peut calculer le % de variance de chaque axe 
    if pca : 
        v1 = str(round(100*pca.explained_variance_ratio_[x]))  + " %"
        v2 = str(round(100*pca.explained_variance_ratio_[y]))  + " %"
    else : 
        v1=v2= ''

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel(f'F{x+1} {v1}')
    ax.set_ylabel(f'F{y+1} {v2}')

    # Valeur x max et y max
    x_max = np.abs(X_[:, x]).max() *1.1
    y_max = np.abs(X_[:, y]).max() *1.1

    # On borne x et y 
    ax.set_xlim(left=-x_max, right=x_max)
    ax.set_ylim(bottom= -y_max, top=y_max)

    # Affichage des lignes horizontales et verticales
    plt.plot([-x_max, x_max], [0, 0], color='grey', alpha=0.8)
    plt.plot([0,0], [-y_max, y_max], color='grey', alpha=0.8)

    # Affichage des labels des points
    if len(labels) : 
        # j'ai copié collé la fonction sans la lire
        for i,(_x,_y) in enumerate(X_[:,[x,y]]):
            plt.text(_x, _y+0.05, labels[i], fontsize='14', ha='center',va='center') 

    # Titre et display
    plt.title(f"Projection des individus (sur F{x+1} et F{y+1})")
    plt.show()

## Import data

In [10]:
imported_data=pd.read_csv("/Users/souha_kassab/OC_project3/static/csv/2016_Building_Energy_Benchmarking.csv", decimal='.', low_memory=False)

In [11]:
## Check dtypes & NaN Values 
data_desc(imported_data)

(3376, 46)
(3376, 46)


OSEBuildingID                        int64
DataYear                             int64
BuildingType                        object
PrimaryPropertyType                 object
PropertyName                        object
Address                             object
City                                object
State                               object
ZipCode                            float64
TaxParcelIdentificationNumber       object
CouncilDistrictCode                  int64
Neighborhood                        object
Latitude                           float64
Longitude                          float64
YearBuilt                            int64
NumberofBuildings                  float64
NumberofFloors                       int64
PropertyGFATotal                     int64
PropertyGFAParking                   int64
PropertyGFABuilding(s)               int64
ListOfAllPropertyUseTypes           object
LargestPropertyUseType              object
LargestPropertyUseTypeGFA          float64
SecondLarge

# Data description

## First glance

In [12]:
imported_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [13]:
# Formating columns titles
columns_title_format(imported_data)

['Osebuildingid', 'Datayear', 'Buildingtype', 'Primarypropertytype', 'Propertyname', 'Address', 'City', 'State', 'Zipcode', 'Taxparcelidentificationnumber', 'Councildistrictcode', 'Neighborhood', 'Latitude', 'Longitude', 'Yearbuilt', 'Numberofbuildings', 'Numberoffloors', 'Propertygfatotal', 'Propertygfaparking', 'Propertygfabuilding(S)', 'Listofallpropertyusetypes', 'Largestpropertyusetype', 'Largestpropertyusetypegfa', 'Secondlargestpropertyusetype', 'Secondlargestpropertyusetypegfa', 'Thirdlargestpropertyusetype', 'Thirdlargestpropertyusetypegfa', 'Yearsenergystarcertified', 'Energystarscore', 'Siteeui(Kbtu/Sf)', 'Siteeuiwn(Kbtu/Sf)', 'Sourceeui(Kbtu/Sf)', 'Sourceeuiwn(Kbtu/Sf)', 'Siteenergyuse(Kbtu)', 'Siteenergyusewn(Kbtu)', 'Steamuse(Kbtu)', 'Electricity(Kwh)', 'Electricity(Kbtu)', 'Naturalgas(Therms)', 'Naturalgas(Kbtu)', 'Defaultdata', 'Comments', 'Compliancestatus', 'Outlier', 'Totalghgemissions', 'Ghgemissionsintensity']


,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Comments,Compliancestatus,Outlier,Totalghgemissions,Ghgemissionsintensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,NaN,Compliant,NaN,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,...,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,...,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,...,5.252517e+05,1.792159e+06,39737.390630,3.973739e+06,False,NaN,Compliant,NaN,223.54,16.99
3374,50225,2016,Nonresidential COS,Mixed Use Property,South Park Community Center,8319 8th Ave S,Seattle,WA,NaN,7883603155,...,1.022480e+05,3.488702e+05,3706.010010,3.706010e+05,False,NaN,Compliant,NaN,22.11,1.57


In [14]:
## How many unique values per entry?
imported_data.nunique()

Osebuildingid                      3376
Datayear                              1
Buildingtype                          8
Primarypropertytype                  24
Propertyname                       3362
Address                            3354
City                                  1
State                                 1
Zipcode                              55
Taxparcelidentificationnumber      3268
Councildistrictcode                   7
Neighborhood                         19
Latitude                           2876
Longitude                          2656
Yearbuilt                           113
Numberofbuildings                    17
Numberoffloors                       50
Propertygfatotal                   3195
Propertygfaparking                  496
Propertygfabuilding(S)             3193
Listofallpropertyusetypes           466
Largestpropertyusetype               56
Largestpropertyusetypegfa          3122
Secondlargestpropertyusetype         50
Secondlargestpropertyusetypegfa    1352


## Data cleaning

In [15]:
## Percentage of Nan values per variable
display_nan_Values(imported_data)

{'Comments': 1.0,
 'Outlier': 0.990521327014218,
 'Yearsenergystarcertified': 0.9647511848341233,
 'Thirdlargestpropertyusetype': 0.8234597156398105,
 'Thirdlargestpropertyusetypegfa': 0.8234597156398105,
 'Secondlargestpropertyusetype': 0.5026658767772512,
 'Secondlargestpropertyusetypegfa': 0.5026658767772512,
 'Energystarscore': 0.2497037914691943,
 'Largestpropertyusetype': 0.005924170616113744,
 'Largestpropertyusetypegfa': 0.005924170616113744,
 'Zipcode': 0.004739336492890996,
 'Listofallpropertyusetypes': 0.002665876777251185,
 'Sourceeui(Kbtu/Sf)': 0.002665876777251185,
 'Sourceeuiwn(Kbtu/Sf)': 0.002665876777251185,
 'Steamuse(Kbtu)': 0.002665876777251185,
 'Electricity(Kwh)': 0.002665876777251185,
 'Electricity(Kbtu)': 0.002665876777251185,
 'Naturalgas(Therms)': 0.002665876777251185,
 'Naturalgas(Kbtu)': 0.002665876777251185,
 'Totalghgemissions': 0.002665876777251185,
 'Ghgemissionsintensity': 0.002665876777251185,
 'Numberofbuildings': 0.002369668246445498,
 'Siteeui(Kbtu/

In [16]:
[nan_data, data_cols] = compute_nan_P(imported_data)

Thirdlargestpropertyusetype
Thirdlargestpropertyusetypegfa
Yearsenergystarcertified
Comments
Outlier


In [17]:
## How many duplicates?
imported_data[imported_data.duplicated()].shape

(0, 46)

In [18]:
## Drop nan columns
first_dataset = imported_data.iloc[:, data_cols]
data_desc(first_dataset)
# -------------------------------------------------------------------------------

(3376, 41)
(3376, 41)


Osebuildingid                        int64
Datayear                             int64
Buildingtype                        object
Primarypropertytype                 object
Propertyname                        object
Address                             object
City                                object
State                               object
Zipcode                            float64
Taxparcelidentificationnumber       object
Councildistrictcode                  int64
Neighborhood                        object
Latitude                           float64
Longitude                          float64
Yearbuilt                            int64
Numberofbuildings                  float64
Numberoffloors                       int64
Propertygfatotal                     int64
Propertygfaparking                   int64
Propertygfabuilding(S)               int64
Listofallpropertyusetypes           object
Largestpropertyusetype              object
Largestpropertyusetypegfa          float64
Secondlarge

In [19]:
first_dataset.shape

(3376, 41)

## Data formatting

In [38]:
# Function for replacing space with '_' in columns of a df
def replace_space(dataset):
    df = dataset.select_dtypes(include=['object'])
    df = df.stack().str.replace(' ', '_').unstack()
    #dataset.loc[:, df.columns] = df[:]
    # modif du 8/2/2023
    #dataset.B = df1.E
    for col in df.columns:
        dataset[col] = df[col]
    
    #del df
    return dataset

In [26]:
replace_space(first_dataset)
#first_dataset = replace_space(first_dataset)
first_dataset.head()

/var/folders/xx/m7dpq0_56k9crnft1k608cb80000gn/T/ipykernel_1468/4171256986.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,col] = df[col]


,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
0,1,2016,NonResidential,Hotel,Mayflower_park_hotel,405_Olive_way,Seattle,WA,98101.0,0659000030,...,7456910.0,2003882.00,1.156514e+06,3946027.0,12764.52930,1276453.0,False,Compliant,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount_Hotel,724_Pine_street,Seattle,WA,98101.0,0659000220,...,8664479.0,0.00,9.504252e+05,3242851.0,51450.81641,5145082.0,False,Compliant,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The_Westin_Seattle,1900_5th_Avenue,Seattle,WA,98101.0,0659000475,...,73937112.0,21566554.00,1.451544e+07,49526664.0,14938.00000,1493800.0,False,Compliant,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL_MAX,620_STEWART_ST,Seattle,WA,98101.0,0659000640,...,6946800.5,2214446.25,8.115253e+05,2768924.0,18112.13086,1811213.0,False,Compliant,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK_SEATTLE_HOTEL_(ID8),401_LENORA_ST,Seattle,WA,98121.0,0659000970,...,14656503.0,0.00,1.573449e+06,5368607.0,88039.98438,8803998.0,False,Compliant,505.01,2.88


In [27]:
first_dataset.describe(include='all')

,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
count,3376.000000,3376.0,3376,3376,3376,3376,3376,3376,3360.000000,3376,...,3.370000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3376,3376,3367.000000,3367.000000
unique,NaN,NaN,8,24,3362,3354,1,1,NaN,3268,...,NaN,NaN,NaN,NaN,NaN,NaN,2,4,NaN,NaN
top,NaN,NaN,NonResidential,Low-Rise_Multifamily,Northgate_Plaza,2600_SW_Barton_St,Seattle,WA,NaN,1625049001,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Compliant,NaN,NaN
freq,NaN,NaN,1460,987,3,4,3376,3376,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,3263,3211,NaN,NaN
mean,21208.991114,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98116.949107,NaN,...,5.276726e+06,2.745959e+05,1.086639e+06,3.707612e+06,1.368505e+04,1.368505e+06,NaN,NaN,119.723971,1.175916
std,12223.757015,0.0,NaN,NaN,NaN,NaN,NaN,NaN,18.615205,NaN,...,1.593879e+07,3.912173e+06,4.352478e+06,1.485066e+07,6.709781e+04,6.709781e+06,NaN,NaN,538.832227,1.821452
min,1.000000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98006.000000,NaN,...,0.000000e+00,0.000000e+00,-3.382680e+04,-1.154170e+05,0.000000e+00,0.000000e+00,NaN,NaN,-0.800000,-0.020000
25%,19990.750000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98105.000000,NaN,...,9.701822e+05,0.000000e+00,1.874229e+05,6.394870e+05,0.000000e+00,0.000000e+00,NaN,NaN,9.495000,0.210000
50%,23112.000000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98115.000000,NaN,...,1.904452e+06,0.000000e+00,3.451299e+05,1.177583e+06,3.237538e+03,3.237540e+05,NaN,NaN,33.920000,0.610000
75%,25994.250000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98122.000000,NaN,...,4.381429e+06,0.000000e+00,8.293178e+05,2.829632e+06,1.189033e+04,1.189034e+06,NaN,NaN,93.940000,1.370000


In [ ]:
modif = first_dataset.copy()
df = modif.select_dtypes(include=['object'])
df = df.stack().str.replace(' ', '_').unstack()
df.columns
replace_space(modif)

## Saving first-pass clean data

In [ ]:
first_dataset.to_csv('/Users/souha_kassab/OC_project3/static/csv/first_clean_dataset_3_2.csv')

Data preparation 

# Data preparation

In [28]:
first_dataset.shape

(3376, 41)

In [29]:
first_dataset.isna().sum()

Osebuildingid                         0
Datayear                              0
Buildingtype                          0
Primarypropertytype                   0
Propertyname                          0
Address                               0
City                                  0
State                                 0
Zipcode                              16
Taxparcelidentificationnumber         0
Councildistrictcode                   0
Neighborhood                          0
Latitude                              0
Longitude                             0
Yearbuilt                             0
Numberofbuildings                     8
Numberoffloors                        0
Propertygfatotal                      0
Propertygfaparking                    0
Propertygfabuilding(S)                0
Listofallpropertyusetypes             9
Largestpropertyusetype               20
Largestpropertyusetypegfa            20
Secondlargestpropertyusetype       1697
Secondlargestpropertyusetypegfa    1697


Discarding un-retrievable data (missing source consumption and emisison) : 12 values

In [30]:
nan_energy=first_dataset[(first_dataset['Siteenergyuse(Kbtu)'].isna())
                                 & (first_dataset['Totalghgemissions'].isna())]

first_dataset.loc[nan_energy.index]

,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
578,773,2016,NonResidential,Small-_and_Mid-Sized_Office,SEATTLE_BUILDING,215_COLUMBIA_ST,Seattle,WA,98104.0,0939000245,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Non-Compliant,NaN,NaN
758,19798,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,SANDHURST_APTS,7239_SAND_POINT_WAY_NE,Seattle,WA,98115.0,0225049032,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Non-Compliant,NaN,NaN
1792,23437,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,MERCER_CREST_APT,500_W_MERCER_ST,Seattle,WA,98119.0,3876900230,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Non-Compliant,NaN,NaN
2474,25752,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,GRANVIEW_APARTMENTS,1120_N_93RD_ST,Seattle,WA,98103.0,9267200155,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Non-Compliant,NaN,NaN
2670,26532,2016,NonResidential,Mixed_Use_Property,KALBERG_BUILDING,4515_UNIVERSITY_WAY_NE,Seattle,WA,98105.0,8816401120,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Non-Compliant,NaN,NaN


In [31]:
null_energy=first_dataset[(first_dataset['Siteenergyuse(Kbtu)']==0)
                                 & (first_dataset['Totalghgemissions']==0)]

first_dataset.loc[null_energy.index]

,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
28,34,2016,SPS-District_K-12,K-12_School,Meany_Building,301/300_21st_Ave._E.,Seattle,WA,98112.0,0688000090,...,0.0,0.0,0.0,0.0,0.0,0.0,False,Missing_Data,0.0,0.0
152,242,2016,SPS-District_K-12,K-12_School,Olympic_View_Elementary,504_N.E._95th_St.,Seattle,WA,98115.0,5100404840,...,0.0,0.0,0.0,0.0,0.0,0.0,False,Missing_Data,0.0,0.0
304,435,2016,NonResidential,Other,Washington_State_Convention_Center,705_Pike_St,Seattle,WA,98101.0,1978200105,...,0.0,0.0,0.0,0.0,0.0,0.0,False,Non-Compliant,0.0,0.0
746,19776,2016,NonResidential,Other,Welcome_Home_Society_-_PriceCo,13537_Aurora_Ave_N,Seattle,WA,98133.0,0164000020,...,0.0,0.0,0.0,0.0,0.0,0.0,False,Missing_Data,0.0,0.0
760,19801,2016,Multifamily_LR_(1-4),Other,APEX_BELLTOWN_COOP,2225_First_Ave,Seattle,WA,98121.0,0246300000,...,0.0,0.0,0.0,0.0,0.0,0.0,False,Missing_Data,0.0,0.0
1361,21616,2016,SPS-District_K-12,K-12_School,Olympic_Hills_Elementary,13018_20th_Ave._N.E.,Seattle,WA,98125.0,2126049041,...,0.0,0.0,0.0,0.0,0.0,0.0,True,Error_-_Correct_Default_Data,0.0,0.0
1549,22363,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,Forte_Condominium,840_NE_125th_St,Seattle,WA,98125.0,2607980000,...,0.0,0.0,0.0,0.0,0.0,0.0,False,Non-Compliant,0.0,0.0


In [32]:
first_dataset = droping_nan_variable('Siteenergyuse(Kbtu)',first_dataset)

In [33]:
first_dataset = first_dataset.drop(null_energy.index, axis=0)

In [34]:
first_dataset.isna().sum()

Osebuildingid                         0
Datayear                              0
Buildingtype                          0
Primarypropertytype                   0
Propertyname                          0
Address                               0
City                                  0
State                                 0
Zipcode                              16
Taxparcelidentificationnumber         0
Councildistrictcode                   0
Neighborhood                          0
Latitude                              0
Longitude                             0
Yearbuilt                             0
Numberofbuildings                     3
Numberoffloors                        0
Propertygfatotal                      0
Propertygfaparking                    0
Propertygfabuilding(S)                0
Listofallpropertyusetypes             4
Largestpropertyusetype               15
Largestpropertyusetypegfa            15
Secondlargestpropertyusetype       1687
Secondlargestpropertyusetypegfa    1687


In [26]:
first_dataset.describe(include='all')

,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
count,3364.000000,3364.0,3364,3364,3364,3364,3364,3364,3348.000000,3364,...,3.363000e+03,3.360000e+03,3.360000e+03,3.360000e+03,3.360000e+03,3.360000e+03,3364,3364,3360.000000,3360.000000
unique,NaN,NaN,8,24,3350,3342,1,1,NaN,3256,...,NaN,NaN,NaN,NaN,NaN,NaN,2,4,NaN,NaN
top,NaN,NaN,NonResidential,Low-Rise Multifamily,Northgate Plaza,2600 SW Barton St,Seattle,WA,NaN,1625049001,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Compliant,NaN,NaN
freq,NaN,NaN,1456,983,3,4,3364,3364,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,3252,3211,NaN,NaN
mean,21230.973543,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98116.956691,NaN,...,5.287709e+06,2.751680e+05,1.088903e+06,3.715336e+06,1.371356e+04,1.371356e+06,NaN,NaN,119.973396,1.178366
std,12223.669104,0.0,NaN,NaN,NaN,NaN,NaN,NaN,18.638933,NaN,...,1.595355e+07,3.916228e+06,4.356728e+06,1.486516e+07,6.716477e+04,6.716477e+06,NaN,NaN,539.365637,1.822557
min,1.000000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98006.000000,NaN,...,0.000000e+00,0.000000e+00,-3.382680e+04,-1.154170e+05,0.000000e+00,0.000000e+00,NaN,NaN,-0.800000,-0.020000
25%,20004.750000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98105.000000,NaN,...,9.793626e+05,0.000000e+00,1.882482e+05,6.423028e+05,0.000000e+00,0.000000e+00,NaN,NaN,9.580000,0.210000
50%,23118.500000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98115.000000,NaN,...,1.913404e+06,0.000000e+00,3.461780e+05,1.181159e+06,3.253740e+03,3.253740e+05,NaN,NaN,34.025000,0.620000
75%,25996.250000,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,98122.000000,NaN,...,4.387392e+06,0.000000e+00,8.303788e+05,2.833252e+06,1.190571e+04,1.190571e+06,NaN,NaN,94.070000,1.380000


Negative values on electricity consumption, Total emissions and GHGemissions intensity

In [35]:
first_dataset[first_dataset['Siteenergyuse(Kbtu)'].isna()==True]

,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity


In [36]:
first_dataset[first_dataset['Totalghgemissions'].isna()==True]

,Osebuildingid,Datayear,Buildingtype,Primarypropertytype,Propertyname,Address,City,State,Zipcode,Taxparcelidentificationnumber,...,Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kwh),Electricity(Kbtu),Naturalgas(Therms),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
1768,23355,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,GRAHAM_VIEW,6040_CALIFORNIA_AVE_SW,Seattle,WA,98136.0,7625703280,...,5.937986e+05,NaN,NaN,NaN,NaN,NaN,False,Compliant,NaN,NaN
2321,25431,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,PONDERAY_APTS,5625_15TH_AVE_NE,Seattle,WA,98105.0,5226300030,...,7.975070e+05,NaN,NaN,NaN,NaN,NaN,False,Compliant,NaN,NaN
2480,25763,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,Villa_Andora,1520_CALIFORNIA_AVE_SW,Seattle,WA,98116.0,9272201340,...,1.126782e+06,NaN,NaN,NaN,NaN,NaN,False,Compliant,NaN,NaN
3348,50082,2016,Multifamily_LR_(1-4),Low-Rise_Multifamily,STREAM_FIFTEEN,605_15TH_AVE_E,Seattle,WA,98112.0,3303700030,...,8.576127e+05,NaN,NaN,NaN,NaN,NaN,False,Compliant,NaN,NaN


## Drop irrelevant and repeated features 
(same value for all samples)(same units)

In [37]:
first_dataset.nunique()

Osebuildingid                      3364
Datayear                              1
Buildingtype                          8
Primarypropertytype                  24
Propertyname                       3350
Address                            3342
City                                  1
State                                 1
Zipcode                              55
Taxparcelidentificationnumber      3256
Councildistrictcode                   7
Neighborhood                         19
Latitude                           2869
Longitude                          2646
Yearbuilt                           113
Numberofbuildings                    17
Numberoffloors                       50
Propertygfatotal                   3184
Propertygfaparking                  495
Propertygfabuilding(S)             3181
Listofallpropertyusetypes           464
Largestpropertyusetype               55
Largestpropertyusetypegfa          3115
Secondlargestpropertyusetype         50
Secondlargestpropertyusetypegfa    1350


In [39]:
first_dataset['Defaultdata'].unique()

array([False,  True])

In [40]:
#first_dataset.drop(columns=['Latitude','Longitude'], inplace=True)
first_dataset.drop(columns=['Datayear', 'City', 'State', 'Electricity(Kwh)', 
                           'Naturalgas(Therms)'], inplace=True)

In [41]:
first_dataset.drop(columns=['Taxparcelidentificationnumber', 'Councildistrictcode'],
                   inplace=True)

In [42]:
first_dataset.shape

(3364, 34)

In [43]:
def unify_objectvariables_case(dataset):
    object_cols = dataset.select_dtypes(include=['object'])
    for col in object_cols.columns:
        dataset[col] = dataset[col].str.lower()
    print('Case-formatted data shape is:', dataset.shape)
    return dataset

In [44]:
first_dataset = unify_objectvariables_case(first_dataset)
first_dataset.head()

Case-formatted data shape is: (3364, 34)


,Osebuildingid,Buildingtype,Primarypropertytype,Propertyname,Address,Zipcode,Neighborhood,Latitude,Longitude,Yearbuilt,...,Sourceeuiwn(Kbtu/Sf),Siteenergyuse(Kbtu),Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kbtu),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
0,1,nonresidential,hotel,mayflower_park_hotel,405_olive_way,98101.0,downtown,47.61220,-122.33799,1927,...,189.000000,7226362.5,7456910.0,2003882.00,3946027.0,1276453.0,False,compliant,249.98,2.83
1,2,nonresidential,hotel,paramount_hotel,724_pine_street,98101.0,downtown,47.61317,-122.33393,1996,...,179.399994,8387933.0,8664479.0,0.00,3242851.0,5145082.0,False,compliant,295.86,2.86
2,3,nonresidential,hotel,5673-the_westin_seattle,1900_5th_avenue,98101.0,downtown,47.61393,-122.33810,1969,...,244.100006,72587024.0,73937112.0,21566554.00,49526664.0,1493800.0,False,compliant,2089.28,2.19
3,5,nonresidential,hotel,hotel_max,620_stewart_st,98101.0,downtown,47.61412,-122.33664,1926,...,224.000000,6794584.0,6946800.5,2214446.25,2768924.0,1811213.0,False,compliant,286.43,4.67
4,8,nonresidential,hotel,warwick_seattle_hotel_(id8),401_lenora_st,98121.0,downtown,47.61375,-122.34047,1980,...,215.600006,14172606.0,14656503.0,0.00,5368607.0,8803998.0,False,compliant,505.01,2.88


In [45]:
first_dataset.nunique()

Osebuildingid                      3364
Buildingtype                          8
Primarypropertytype                  24
Propertyname                       3348
Address                            3341
Zipcode                              55
Neighborhood                         14
Latitude                           2869
Longitude                          2646
Yearbuilt                           113
Numberofbuildings                    17
Numberoffloors                       50
Propertygfatotal                   3184
Propertygfaparking                  495
Propertygfabuilding(S)             3181
Listofallpropertyusetypes           464
Largestpropertyusetype               55
Largestpropertyusetypegfa          3115
Secondlargestpropertyusetype         50
Secondlargestpropertyusetypegfa    1350
Energystarscore                     100
Siteeui(Kbtu/Sf)                   1085
Siteeuiwn(Kbtu/Sf)                 1105
Sourceeui(Kbtu/Sf)                 1648
Sourceeuiwn(Kbtu/Sf)               1694


In [ ]:
first_dataset[first_dataset['Electricity(Kbtu)']<0].loc[:,'Siteenergyuse(Kbtu)':]
first_dataset.drop(3206, axis=0, inplace=True)

In [ ]:
first_dataset.shape

In [ ]:
first_dataset.duplicated().sum()

## Handle Nans

In [53]:
# function with condition that if List has only one element secondproperty is null
first_dataset[first_dataset['Secondlargestpropertyusetype'].isna()]['Listofallpropertyusetypes']

0                    hotel
2                    hotel
3                    hotel
5           police_station
6                    hotel
               ...        
3367    other_-_recreation
3369                office
3370    other_-_recreation
3371                office
3372    other_-_recreation
Name: Listofallpropertyusetypes, Length: 1687, dtype: object

In [54]:
first_dataset['Secondlargestpropertyusetype'].fillna(value = 'none', inplace=True)
first_dataset['Secondlargestpropertyusetypegfa'].fillna(value = '0', inplace=True)

In [55]:
first_dataset[first_dataset['Secondlargestpropertyusetype']=='none']['Secondlargestpropertyusetypegfa']

0       0
2       0
3       0
5       0
6       0
       ..
3367    0
3369    0
3370    0
3371    0
3372    0
Name: Secondlargestpropertyusetypegfa, Length: 1687, dtype: object

In [56]:
first_dataset['Listofallpropertyusetypes'].unique()

array(['hotel', 'hotel,_parking,_restaurant',
       'hotel,_parking,_swimming_pool', 'police_station',
       'other_-_entertainment/public_assembly', 'multifamily_housing',
       'hotel,_parking', 'data_center,_library,_parking',
       'fitness_center/health_club/gym,_office,_swimming_pool',
       'office,_other,_other_-_lodging/residential,_restaurant,_social/meeting_hall',
       'courthouse', 'other', 'k-12_school', 'college/university',
       'automobile_dealership',
       'hotel,_parking,_retail_store,_swimming_pool',
       'k-12_school,_multifamily_housing,_office,_other_-_entertainment/public_assembly',
       'self-storage_facility',
       'energy/power_station,_laboratory,_manufacturing/industrial_plant,_non-refrigerated_warehouse,_office',
       'non-refrigerated_warehouse,_office,_refrigerated_warehouse',
       'non-refrigerated_warehouse,_office', 'office,_other_-_education',
       'office,_self-storage_facility,_vocational_school',
       'data_center,_office,_

In [ ]:
first_dataset['Listofallpropertyusetypes'].split('/')

In [71]:
dropindex=first_dataset.loc[first_dataset['Largestpropertyusetype'].isna()]
dropindex.index

Int64Index([ 353, 1077, 1147, 1483, 1756, 1768, 2321, 2336, 2414, 2459, 2480,
            2545, 2886, 3040, 3348],
           dtype='int64')

In [72]:
first_dataset.drop(dropindex.index, inplace=True)

In [73]:
first_dataset.shape

(3349, 34)

In [75]:
display_nan_Values(first_dataset)

{'Energystarscore': 0.24604359510301582,
 'Zipcode': 0.00477754553598089,
 'Siteeuiwn(Kbtu/Sf)': 0.00029859659599880563,
 'Siteenergyusewn(Kbtu)': 0.00029859659599880563,
 'Osebuildingid': 0.0,
 'Buildingtype': 0.0,
 'Primarypropertytype': 0.0,
 'Propertyname': 0.0,
 'Address': 0.0,
 'Neighborhood': 0.0,
 'Latitude': 0.0,
 'Longitude': 0.0,
 'Yearbuilt': 0.0,
 'Numberofbuildings': 0.0,
 'Numberoffloors': 0.0,
 'Propertygfatotal': 0.0,
 'Propertygfaparking': 0.0,
 'Propertygfabuilding(S)': 0.0,
 'Listofallpropertyusetypes': 0.0,
 'Largestpropertyusetype': 0.0,
 'Largestpropertyusetypegfa': 0.0,
 'Secondlargestpropertyusetype': 0.0,
 'Secondlargestpropertyusetypegfa': 0.0,
 'Siteeui(Kbtu/Sf)': 0.0,
 'Sourceeui(Kbtu/Sf)': 0.0,
 'Sourceeuiwn(Kbtu/Sf)': 0.0,
 'Siteenergyuse(Kbtu)': 0.0,
 'Steamuse(Kbtu)': 0.0,
 'Electricity(Kbtu)': 0.0,
 'Naturalgas(Kbtu)': 0.0,
 'Defaultdata': 0.0,
 'Compliancestatus': 0.0,
 'Totalghgemissions': 0.0,
 'Ghgemissionsintensity': 0.0}

## Handle outliers >> analyse explo

In [ ]:
first_dataset['Longitude'].max()

In [ ]:
first_dataset[first_dataset['Zipcode']==first_dataset['Zipcode'].max()]

In [ ]:
first_dataset[first_dataset['Longitude']==first_dataset['Longitude'].max()]

In [ ]:
first_dataset.loc[first_dataset['Propertygfatotal']==first_dataset['Propertygfatotal'].max(),:]['Propertygfatotal']

In [ ]:
first_dataset.loc[first_dataset['Propertygfaparking']==first_dataset['Propertygfaparking'].max(),:]

In [32]:
first_dataset.loc[first_dataset['Numberoffloors']==first_dataset['Numberoffloors'].max(),:]

,Osebuildingid,Buildingtype,Primarypropertytype,Propertyname,Address,Zipcode,Neighborhood,Latitude,Longitude,Yearbuilt,...,Sourceeuiwn(Kbtu/Sf),Siteenergyuse(Kbtu),Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kbtu),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
1359,21611,nonresidential,worship_facility,seattle_chinese_baptist_church,5801_beacon_avenue_south,98108.0,greater_duwamish,47.55072,-122.30265,1977,...,46.599998,326001.1875,326001.1875,0.0,326001.0,0.0,False,compliant,2.27,0.1


In [34]:
first_dataset.drop(index=1359,inplace=True)

In [35]:
first_dataset.loc[first_dataset['Totalghgemissions']==first_dataset['Totalghgemissions'].max(),:]

,Osebuildingid,Buildingtype,Primarypropertytype,Propertyname,Address,Zipcode,Neighborhood,Latitude,Longitude,Yearbuilt,...,Sourceeuiwn(Kbtu/Sf),Siteenergyuse(Kbtu),Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kbtu),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
35,43,campus,mixed_use_property,plant_2_site,7755_e_marginal_way_s,98108.0,greater_duwamish,47.53493,-122.31788,1992,...,400.299988,448385312.0,471613856.0,0.0,150476283.0,297909000.0,False,compliant,16870.98,34.09


In [36]:
first_dataset.loc[first_dataset['Siteenergyuse(Kbtu)']==first_dataset['Siteenergyuse(Kbtu)'].max(),:]

,Osebuildingid,Buildingtype,Primarypropertytype,Propertyname,Address,Zipcode,Neighborhood,Latitude,Longitude,Yearbuilt,...,Sourceeuiwn(Kbtu/Sf),Siteenergyuse(Kbtu),Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kbtu),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity
3274,49967,campus,university,university_of_washington_-_seattle_campus,ne_45th_st_&_brooklyn_ave,98105.0,northeast,47.65644,-122.31041,1900,...,0.0,873923712.0,0.0,84985240.0,657074389.0,0.0,False,compliant,11140.56,1.2


In [ ]:
first_dataset.drop(index=35)

In [37]:
 #first_dataset[first_dataset['Compliancestatus'] == 'Missing_Data']
null_energy=first_dataset[(first_dataset['Siteenergyuse(Kbtu)']==0)
                                 & (first_dataset['Totalghgemissions']==0)]

first_dataset.loc[null_energy.index]

,Osebuildingid,Buildingtype,Primarypropertytype,Propertyname,Address,Zipcode,Neighborhood,Latitude,Longitude,Yearbuilt,...,Sourceeuiwn(Kbtu/Sf),Siteenergyuse(Kbtu),Siteenergyusewn(Kbtu),Steamuse(Kbtu),Electricity(Kbtu),Naturalgas(Kbtu),Defaultdata,Compliancestatus,Totalghgemissions,Ghgemissionsintensity


In [83]:
first_dataset[first_dataset['Siteeuiwn(Kbtu/Sf)'].isna()]['Siteenergyusewn(Kbtu)']

563   NaN
Name: Siteenergyusewn(Kbtu), dtype: float64

In [84]:
first_dataset.drop(index=563, inplace=True)

------------------------------------------------------------------------------------------

In [76]:
first_dataset.duplicated().sum()

0

In [85]:
first_dataset.shape

(3348, 34)

    Traitement spécifique par la suite pour les valeurs NaN en transmission ou emission
   ------------------------------------------------------------------------------------

## Saving second-pass clean data for separate analysis of emissions and consumption

In [86]:
first_dataset.to_csv('/Users/souha_kassab/OC_project3/static/csv/second_clean_dataset_20230903.csv')

In [ ]:
first_dataset.groupby('Numberoffloors').mean()
pd.crosstab(Year_projection['Yearbuilt'], Total_GHG_emission_projection.bldg_total_emi, dropna=True)

# Statistical description

## Total GHG emissions analysis

### Barplot

In [ ]:
## barplot of a Property main use type after discarding residential buildings (multifamily)
count_qualiVar = first_dataset['Primarypropertytype'].value_counts()
#count_qualiVar.values.sort()
#count_qualiVar.index
ax = sns.barplot(y=count_qualiVar.index, x=count_qualiVar.values)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
# Countplot
plt.figure()
plt.subplot(111)
sns.countplot(data=first_dataset, x ='Primarypropertytype', hue= 'Compliancestatus')
plt.xticks(rotation=90)
plt.title('Property primary use and compliance status')
plt.show()

In [ ]:
## Plotting the distribution of data over compliance status 
plt.figure(figsize=(5,3))
first_dataset['Compliancestatus'].value_counts(normalize=True).plot(kind='bar',width=0.1)
plt.show()

### Displot

In [ ]:
x = np.log(outliers_ghgemissionintensity[outliers_ghgemissionintensity.Totalghgemissions > 0]["Totalghgemissions"])

In [ ]:
# -------------------------------------------------------------------------------
# 5.2.2. Representing continuous variables using histograms
plt.figure(figsize=(8,8))
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
sns.displot(x, kde=True)
plt.show()
# -------------------------------------------------------------------------------

In [ ]:
def is_family(item):
    return "multifamily" in item or "Multifamily" in item

In [ ]:
first_dataset[first_dataset['Primarypropertytype'].map(is_family)].index

In [ ]:
indextodrop = first_dataset[first_dataset['Primarypropertytype'].map(is_family)].index
first_dataset.drop(indextodrop, axis=0, inplace=True)
print(first_dataset.shape)

In [ ]:
len(first_dataset)/len(imported_data)*100

In [ ]:
x = np.log(first_dataset[first_dataset.Totalghgemissions > 0]["Totalghgemissions"])

In [ ]:
# -------------------------------------------------------------------------------
# 5.2.2. Representing continuous variables using histograms
plt.figure(figsize=(8,8))
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
sns.displot(x, kde=True)
plt.show()
# -------------------------------------------------------------------------------

In [ ]:
x_ghg_int = np.log(first_dataset[first_dataset.Ghgemissionsintensity > 1]["Ghgemissionsintensity"])

In [ ]:
# -------------------------------------------------------------------------------
# 5.2.2. Representing continuous variables using histograms
plt.figure(figsize=(8,8))
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
sns.displot(x_ghg_int, kde=True)
plt.show()
# -------------------------------------------------------------------------------

In [ ]:
## Projection on a table for study of empirical distribution
Total_GHG_emission_projection = present_variable_in_tab(first_dataset, 
                                                'Totalghgemissions', 'bldg_total_emi')
Total_GHG_emission_projection.index = np.arange(1,len(Total_GHG_emission_projection)+1)
Total_GHG_emission_projection

In [ ]:
first_dataset[first_dataset['Totalghgemissions']<=1]

### Boxplot

In [ ]:
data_total_ghg_em = pd.DataFrame()
type(data_total_ghg_em)

In [ ]:
first_dataset[ 'Totalghgemissions']

In [ ]:
x = np.log(first_dataset[first_dataset.Totalghgemissions > 0]["Totalghgemissions"])
# -------------------------------------------------------------------------------
sns.boxplot(x='Primarypropertytype', y="Totalghgemissions", data=first_dataset, 
               showfliers = False)
plt.xticks(rotation=90)
plt.show()

In [ ]:
type(x)

In [ ]:
sns.set(style='white', color_codes=True)
sns.jointplot(data=first_dataset[first_dataset['Sourceeui(Kbtu/Sf)']<=1000], x='Sourceeui(Kbtu/Sf)',
              y='Ghgemissionsintensity', kind = 'reg', line_kws={'color': 'red'})
plt.show()

## Energy consumption Analysis

In [ ]:
first_dataset.columns

### Barplot

In [ ]:
## barplot of a Property main use type after discarding residential buildings (multifamily)
count_qualiVar = first_dataset['Primarypropertytype'].value_counts()
#count_qualiVar.values.sort()
#count_qualiVar.index
ax = sns.barplot(y=count_qualiVar.index, x=count_qualiVar.values)
ax.bar_label(ax.containers[0])
plt.show()

### Scatterplot

In [ ]:
##seaborn alternative:
plt.figure()
sns.set_style('darkgrid')
plt.subplot(111)
sns.scatterplot(data=first_dataset, x='Propertygfabuilding(S)', y='Siteenergyuse(Kbtu)',
 hue='Primarypropertytype', style='Buildingtype')
plt.title('Buildings consumption and emissions evolution per years')
plt.legend(loc=0, bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
##seaborn alternative:
plt.figure()
sns.set_style('darkgrid')
plt.subplot(111)
sns.scatterplot(data=first_dataset[first_dataset['Siteenergyuse(Kbtu)']<1e8], x='Propertygfabuilding(S)', y='Siteenergyuse(Kbtu)',
 hue='Primarypropertytype', style='Buildingtype')
plt.title('Buildings consumption and emissions evolution per years')
plt.legend(loc=0, bbox_to_anchor=(1, 1))
plt.show()

### Displot

In [ ]:

# -------------------------------------------------------------------------------
# 5.2.2. Representing continuous variables using histograms
plt.figure()
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
sns.displot(first_dataset['Siteeui(Kbtu/Sf)'], kde=False)
plt.title('Annual amount of energy consumed by property on-site')
plt.show()
# -------------------------------------------------------------------------------


In [ ]:
x_siteui = np.log(first_dataset[first_dataset['Siteenergyuse(Kbtu)']>0]['Siteenergyuse(Kbtu)'])

In [ ]:

# -------------------------------------------------------------------------------
# 5.2.2. Representing continuous variables using histograms
plt.figure()
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
sns.displot(x_siteui, kde=False)
plt.title('Annual amount of energy consumed by property on-site')
plt.show()
# -------------------------------------------------------------------------------

In [ ]:
x_gfa_bld = np.log(first_dataset[first_dataset['Propertygfabuilding(S)']>0]['Propertygfabuilding(S)'])
# -------------------------------------------------------------------------------
# 5.2.2. Representing continuous variables using histograms
plt.figure()
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
sns.displot(x_gfa_bld, kde=False)
plt.title('Annual amount of energy consumed by property on-site')
plt.show()

### Boxplot

In [ ]:

sns.boxplot(y='Primarypropertytype', x="Siteeui(Kbtu/Sf)", data=first_dataset, 
               showfliers = False)
plt.xticks(rotation=90)
plt.show()

In [ ]:
pd.qcut?

### Jointplot

In [ ]:
#log_ghgemissions = np.log(first_dataset.loc[:,'Ghgemissionsintensity'])
sns.set(style='white', color_codes=True)
sns.jointplot(data=first_dataset[first_dataset['Siteenergyuse(Kbtu)']<=1e8], x='Propertygfabuilding(S)',
              y='Siteenergyuse(Kbtu)', kind = 'reg', line_kws={'color': 'red'})
plt.show()

In [ ]:
#log_ghgemissions = np.log(first_dataset.loc[:,'Ghgemissionsintensity'])
sns.set(style='white', color_codes=True)
sns.jointplot(x=x_gfa_bld,
              y=x_siteui, kind = 'reg', line_kws={'color': 'red'})
plt.show()

In [ ]:
## Projection on a table for study of empirical distribution
Total_GHG_emission_projection = present_variable_in_tab(first_dataset, 
                                                'Totalghgemissions', 'bldg_total_emi')
Total_GHG_emission_projection.index = np.arange(1,len(Total_GHG_emission_projection)+1)
Year_projection = present_variable_in_tab(first_dataset, 'Yearbuilt', 
                                                        'Construction_year')
Year_projection.index = np.arange(1,len(Year_projection)+1)

In [ ]:
first_dataset.loc[first_dataset['Totalghgemissions']>10000]

### Pairplot

In [ ]:
## Looking for correlated variables
#pairplot_data = explored_dataset.drop(columns = ['Yearbuilt', 'Prim'])
sns.pairplot(data=explored_dataset, hue = 'Primarypropertytype', 
             vars=['propertygfabuilding(s)', 'totalghgemissions'], height = 2.5)
sns.set_style('ticks')

In [ ]:
## Looking for correlated variables
pairplot_data = explored_dataset.drop(columns = ['Yearbuilt'])
sns.pairplot(data=pairplot_data, hue = 'Primarypropertytype', height = 2.5 )
sns.set_style('ticks')

## Analysis of GHG emissions
### Analysis of central tendencies

In [ ]:
# 5.2. Representing continuous quantitative variables 
# 5.2.1. Central tendency analysis & dispersion measurements
for cat in first_dataset['Compliancestatus'].unique():
    subset = first_dataset.loc[first_dataset['Compliancestatus'] == cat, :] # Création du sous-échantillon
    print("-"*20)
    print(cat)
    print("moy:\n",subset['Ghgemissionsintensity'].mean())
    print("med:\n",subset['Ghgemissionsintensity'].median())
    print("mod:\n",subset['Ghgemissionsintensity'].mode())
    print("var:\n",subset['Ghgemissionsintensity'].var(ddof=0))
    print("ect:\n",subset['Ghgemissionsintensity'].std(ddof=0))
    #subset["Ghgemissionsintensity"].value_counts(normalize=True).plot(kind='bar',width=0.1)
    #plt.show()
    subset.boxplot(column="Ghgemissionsintensity", showfliers = True, vert=False)
    plt.show()


Handling outliers

In [ ]:
outliers_ghgemissionintensity = first_dataset.loc[(first_dataset['Compliancestatus']== 'Compliant') & (first_dataset['Ghgemissionsintensity']>5)]

In [ ]:
del outliers_ghgemissionintensity
compliant_data = first_dataset.loc[(first_dataset['Compliancestatus']== 'Compliant')]
compliant_data.describe()

Drop non-compliant buildings?

In [ ]:
len(outliers_ghgemissionintensity)/len(imported_data)

In [ ]:
## Creating a new dataframe
bld_type_data = first_dataset['Primarypropertytype'].value_counts()[:].keys()
bld_type_data

In [ ]:
donnees_bar = first_dataset.loc[first_dataset['Largestpropertyusetype'].isin(bld_type_data)].groupby([ 'Primarypropertytype', 'Largestpropertyusetype']).size().unstack(1)

In [ ]:
donnees_bar
pos = liste 

In [ ]:
plt.figure(figsize=[15,10])
plt.subplot(111)
sns.stripplot(x='Primarypropertytype', y = x_gfa_bld.me, hue = 'Buildingtype', data = first_dataset)
plt.xticks(rotation=90)
plt.legend(loc=0, bbox_to_anchor=(1, 1))
plt.show()


In [ ]:
#ordre_prptype = first_dataset.groupby('Compliancestatus')['Primarypropertytype'].median().sort_values().index
## Stripplot
plt.figure(figsize=[15,10])
plt.subplot(111)
sns.stripplot(x='Primarypropertytype', y = x, hue = 'Compliancestatus', data = first_dataset)
plt.xticks(rotation=90, size=12)
plt.yticks(size=14)
#plt.ylabel(size=14)
plt.legend(loc=0, bbox_to_anchor=(1, 1))
plt.show()

Le plus d'outlier semblent exister au niveau des 'campus' avec usage principal en tant qu'hopitaux ou universites, sachant que les usages second et tertiaires seraient la meme(a evrifier)
En moyenne la distribution semble se concentrer autour des memes valeurs 

# Exploratory Analysis

## Data selection

Isolating all numerical values related to energy consumption

In [ ]:
# Function to filter out components with positive numeric values
def is_not_nan(item):
    output = (isinstance(item, float) or isinstance(item, int)) and item >= 0
    return output

In [ ]:
df = first_dataset.select_dtypes(include=['int64', 'float64'])
df = df.loc[:, 'Siteenergyuse(Kbtu)':]

In [ ]:
my_cols = df.columns
#['Yearbuilt', 'Sourceeui(Kbtu/Sf)', 'Ghgemissionsintensity', 'Primarypropertytype']

explored_dataset = first_dataset[my_cols]
explored_dataset.describe(include='all')

In [ ]:
explored_dataset= explored_dataset.drop(null_energy.index, axis=0)

In [ ]:
explored_dataset.shape

In [ ]:
explored_dataset.isnull().sum()

In [ ]:
explored_dataset[explored_dataset['Totalghgemissions']<=0.0]

In [ ]:
explored_dataset = explored_dataset.dropna(how='any')

In [ ]:
cond = (first_dataset['Totalghgemissions'].map(is_not_nan)) 

In [ ]:
cond
#df = df.loc[~cond, :]

In [ ]:
len(df)

In [ ]:
explored_dataset[cond]

In [ ]:
#explored_dataset[explored_dataset['Electricity(Kwh)']<0]
explored_dataset.drop(3206, axis=0, inplace=True)

In [ ]:
explored_dataset.describe(include='all')

In [ ]:
explored_dataset[explored_dataset['Siteenergyusewn(Kbtu)'].isnull()]

 # Saving Data for Exploratory analysis

In [ ]:
explored_dataset.to_csv('/Users/souha_kassab/OC_project3/static/csv/explored_dataset_1.csv')

In [ ]:
#first_dataset = pd.read_csv('/Users/souha_kassab/OC_project3/static/csv/first_dataset_numeric.csv', low_memory=False)

# PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Data split

In [ ]:
## Data split
X = explored_dataset.iloc[:, :-1].values
#first_7dataset.iloc[:, -3]
X[:5]

In [ ]:
X.shape

In [ ]:
#columns in features variable
features = explored_dataset.columns[:-1]
features

In [ ]:
# Instantiate scaler
scaler = StandardScaler()

## Scaling

In [ ]:
X_scaled = scaler.fit_transform(X)
idx = ["mean", "std"]
#reduction et centrage
pd.DataFrame(X_scaled).describe().round(2).loc[idx, :]

## ACP instanciation

In [ ]:
#95% of variance
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(X_scaled)
reduced = pca.transform(X_scaled)

## Variance

In [ ]:
pca.explained_variance_ratio_
scree = (pca.explained_variance_ratio_*100).round(2)
print(scree)
scree_cum = scree.cumsum().round()
print(scree_cum)

x_list = range(1, len(scree)+1)
list(x_list)

In [ ]:
##plot
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("rang de l'axe d'inertie")
plt.ylabel("pourcentage d'inertie")
plt.title("Eboulis des valeurs propres")
plt.show(block=False)

## Components

In [ ]:
pcs = pca.components_
pcs

In [ ]:
##Pandas version
pcs = pd.DataFrame(pcs)
#pcs
##Features labels on cols
pcs.columns = features[:]
pcs.index = [f"F{i}" for i in x_list]
pcs.round(2)

## Correlation graph

In [ ]:
##Define graph axis
x_y = (0,1)
correlation_graph(pca, x_y, features)

In [ ]:
##Exploring F2 and F3
correlation_graph(pca, (1,2), features)

In [ ]:
##Exploring F2 and F3
if (pca.n_components == 0.99):
    correlation_graph(pca, (2,3), features)

plus de rejets lors de l'utilisation de gaz naturel et de vapeur (produite en brulant des fossiles)?
F1 <=> GhG emissions

## Projection

Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)
Variables concernees: 'OSEBuildingID', 'Address',  'Neighborhood',  'BuildingType', 'YearBuilt', 'LargestPropertyUseType' ; 
NaturalGas(kBtu): 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions', 'GHGEmissionsIntensity'
Miscellaneous: 'ComplianceStatus', 'DataYear': Whether a property has met energy benchmarking requirements for the current reporting year.

# Linear Regression

In [ ]:
explored_dataset.head()

## Regression with a matrix of variables

In [ ]:
reg_cols = ['Siteenergyuse(Kbtu)', 'Siteenergyusewn(Kbtu)', 'Steamuse(Kbtu)', 
            'Electricity(Kbtu)', 'Naturalgas(Kbtu)', 'Totalghgemissions']

Linear regression with manual SVD, all datas

In [ ]:
linear_model_dataset = explored_dataset[reg_cols]

In [ ]:
#H = explored_dataset
H = linear_model_dataset
b = H.iloc[:, -1] # Total emissions
A = H.iloc[:, :-1] #all other variables
A = np.pad(A, [(0,0), (0,1)], mode='constant', constant_values =1)
U, S, VT = np.linalg.svd(A, full_matrices=False)
xtilde = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b

# plot model
fig=plt.figure()
ax = plt.subplot(121)

plt.plot(np.arange(1, 1711), b, 'x', color ='k', linewidth = 2, label ='Total emissions' )
plt.plot(np.arange(1, 1711), A@xtilde, '--', color = 'b', linewidth = 1.5 , markersize=6,
         label ='Regression line')
plt.xlabel('index')
plt.ylabel('b')
plt.legend()

# plot sorted values
ax2 = fig.add_subplot(122)
sort_ind = np.argsort(H.iloc[:, -1])
b = b.iloc[sort_ind]
plt.plot(np.arange(1, 1711), b, 'x-', color ='k', linewidth = 2, label ='' )
plt.plot(np.arange(1, 1711), A[sort_ind]@xtilde, '--', color = 'b', linewidth = 1.5 , markersize=6,
         label ='Regression line')
plt.legend()
plt.show()

In [ ]:
MAE = sum(abs(b - A[sort_ind]@xtilde))
MAE

In [ ]:
i = np.where(A[sort_ind] == 0)
print("i = {}".format(i))

In [ ]:
explored_dataset.loc[explored_dataset['Totalghgemissions']<0]

In [ ]:
print('b is a:', type(b), '\n'
     'btrain is a :', type(btrain))

In [ ]:
b.iloc[sort_ind]

In [ ]:
sort_ind

In [ ]:
btrain[btrain<0]

## Linear Regression with data separation in test and train set

In [ ]:
del Atrain, btrain, Atest, btest, A, b

Separate data into 4/5th training and 1/5th testing set

In [ ]:
#Re-define variables
H = linear_model_dataset
b = H.iloc[:, -1] # Total emissions
A = H.iloc[:, :-1] #all other variables
A = np.pad(A, [(0,0), (0,1)], mode='constant', constant_values =1)
## define portions of data to separate into
n = int(len(linear_model_dataset)*4/5)

btrain = b.iloc[:n]
Atrain = A[:n]
btest = b.iloc[n:]
Atest = A[n:]

In [ ]:
# -----------------------------------------------------
# Train
[U, S, VT] = np.linalg.svd(Atrain, full_matrices=False)
xtilde_lin = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ btrain
#xtilde2 = np.linalg.pinv(a) * b
ax1 = fig.add_subplot(121)
plt.plot(np.arange(1, n+1), btrain, 'x', color ='r', markersize = 10, label ='Noisy train data' )
plt.plot(np.arange(1, n+1), Atrain@xtilde_lin, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('row rank')
plt.ylabel('b_train')
plt.legend()
plt.show()
# -----------------------------------------------------
# Test
ax2 = fig.add_subplot(122)
plt.plot(np.arange(1, len(btest)+1), btest, 'x', color ='r', markersize = 10, label ='Noisy test data' )
plt.plot(np.arange(1, len(btest)+1), Atest@xtilde_lin, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('row rank')
plt.ylabel('b_test')
plt.legend()
plt.show()
# -----------------------------------------------------

In [ ]:
sort_ind = np.argsort(btrain)

In [ ]:
# -----------------------------------------------------
# Train
[U, S, VT] = np.linalg.svd(Atrain, full_matrices=False)
xtilde_lin = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ btrain
#xtilde2 = np.linalg.pinv(a) * b
ax1 = fig.add_subplot(121)
plt.plot(np.arange(1, n+1), btrain.iloc[sort_ind], 'x', color ='r', markersize = 10, label ='Noisy train data' )
plt.plot(np.arange(1, n+1), Atrain[sort_ind]@xtilde_lin, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('row rank')
plt.ylabel('b_train')
plt.legend()
plt.show()
# -----------------------------------------------------
# Test
ax2 = fig.add_subplot(122)
plt.plot(np.arange(1, len(btest)+1), btest, 'x', color ='r', markersize = 10, label ='Noisy test data' )
plt.plot(np.arange(1, len(btest)+1), Atest@xtilde_lin, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('row rank')
plt.ylabel('b_test')
plt.legend()
plt.show()
# -----------------------------------------------------

In [ ]:
(btrain.iloc[sort_ind]).head()

Mean Absolute Error

In [ ]:
MAE_test = sum(abs(btest-Atest@xtilde_lin)) / len(btest)
MAE_train = sum(abs(btrain-Atrain@xtilde_lin)) /  len(btrain)
print( 'Mean absolute error on the test set =', MAE_test,
     '\n', 'Mean absolute error on the train set =', MAE_train)
abs(MAE_test - MAE_train)

Root-mean squared error

In [ ]:
RMSE_train = np.sqrt(sum(np.power(btrain-Atrain@xtilde_lin, 2))/len(btrain))
RMSE_train

In [ ]:
RMSE_test = np.sqrt(sum(np.power(btest-Atest@xtilde_lin, 2))/len(btest))
RMSE_test

In [ ]:
plt.plot(np.arange(1, len(btest)+1), (Atest@xtilde_lin-btest), color = 'k',
         linewidth =1)
plt.xlabel('row rank')
plt.ylabel('Error_l1norm')
plt.xticks
plt.show()

In [ ]:
plt.plot(np.arange(1, len(btest)+1), np.power(Atest@xtilde_lin-btest,2), color = 'k',
         linewidth =1)
plt.xlabel('row rank')
plt.ylabel('Error_l2norm')
plt.show()

In [ ]:
n+ np.arange(295,305)

In [ ]:
error_ind = btest.iloc[295:305].index
explored_dataset.loc[error_ind]

## Linear Reg. with prior data shuffling

In [ ]:
del Atrain, btrain, Atest, btest, A, b

In [ ]:
## separate data in train and test sets after shuffling the data
#Re-define variables
H = linear_model_dataset
b = H.iloc[:, -1] # Total emissions
A = H.iloc[:, :-1] #all other variables
A = np.pad(A, [(0,0), (0,1)], mode='constant', constant_values =1)
p = np.random.permutation(len(H))
A = A[p, :]
b = b.iloc[p]
n = int(len(H)*3/5)
btrain = b.iloc[:n]
Atrain = A[:n]
btest = b.iloc[n:]
Atest = A[n:]

In [ ]:
print(n)
print(len(btrain))

In [ ]:
# -----------------------------------------------------
# Train
[U, S, VT] = np.linalg.svd(Atrain, full_matrices=False)
xtilde1 = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ btrain
#xtilde2 = np.linalg.pinv(a) * b
ax1 = fig.add_subplot(121)
plt.plot(np.arange(1, n+1), btrain, 'x', color ='r', markersize = 10, label ='Noisy train data' )
plt.plot(np.arange(1, n+1), Atrain@xtilde1, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('index')
plt.ylabel('b_train')
plt.legend()
#plt.ylim(0, 3000)
plt.show()
# -----------------------------------------------------
# Test
ax2 = fig.add_subplot(122)
plt.plot(np.arange(1, len(btest)+1), btest, 'x', color ='r', markersize = 10, label ='Noisy test data' )
plt.plot(np.arange(1, len(btest)+1), Atest@xtilde1, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('index')
plt.ylabel('b_test')
plt.legend()
#plt.ylim(0, 2000)
plt.show()
# -----------------------------------------------------

In [ ]:
MAE_test = sum(abs(btest-Atest@xtilde1)) / len(btest)
MAE_train = sum(abs(btrain-Atrain@xtilde1)) /  len(btrain)
print( 'Mean absolute error on the test set =', MAE_test,
     '\n', 'Mean absolute error on the train set =', MAE_train)
abs(MAE_test - MAE_train)
#print('Mean absolute error is :', MAE)
# do this for test and for train and compare!

We can see that permuting the dataset prior to constructing the model reduces the MAE difference between train and test set

In [ ]:
RMSE_train = np.sqrt(sum(np.power(btrain-Atrain@xtilde1, 2))/len(btrain))
RMSE_train

In [ ]:
RMSE_test = np.sqrt(sum(np.power(btest-Atest@xtilde1, 2))/len(btest))
RMSE_test

In [ ]:
abs(RMSE_test - RMSE_train)

In [ ]:
np.power(RMSE_train,2)

In [ ]:
y = btrain
y_mean = btrain.mean()
N_samples = len(btrain)
var_mean = sum(np.power(y - y_mean, 2))/N_samples
var_line = np.power(RMSE_train,2)
R2 = (var_mean - var_line)/var_mean
## Dividing all by N: (exactly same computation)
var_mean_bis = y.var()

R2_bis = (var_mean_bis - var_line) / var_mean_bis
print(R2, '\n', R2_bis)

## Linear regression with log-normal

In [ ]:
del Atrain, btrain, Atest, btest, A, b

In [ ]:
H = linear_model_dataset
b = H.iloc[:, -1] # Total emissions
b = np.log(abs(b))
A = H.iloc[:, :-1] #all other variables
A = np.pad(A, [(0,0), (0,1)], mode='constant', constant_values =1)
# Permute
p = np.random.permutation(len(H))
A = A[p, :]
b = b.iloc[p]
# Separate data
n = int(len(H)*3/5)
btrain = b.iloc[:n]
Atrain = A[:n]
btest = b.iloc[n:]
Atest = A[n:]

In [ ]:
# -----------------------------------------------------
# Train
[U, S, VT] = np.linalg.svd(Atrain, full_matrices=False)
xtilde1 = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ btrain
#xtilde2 = np.linalg.pinv(a) * b
ax1 = fig.add_subplot(121)
plt.plot(np.arange(1, n+1), btrain, 'x', color ='r', markersize = 10, label ='Noisy train data' )
plt.plot(np.arange(1, n+1), Atrain@xtilde1, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('index')
plt.ylabel('b_train')
plt.legend()
plt.show()
# -----------------------------------------------------
# Test
ax2 = fig.add_subplot(122)
plt.plot(np.arange(1, len(btest)+1), btest, 'x', color ='r', markersize = 10, label ='Noisy test data' )
plt.plot(np.arange(1, len(btest)+1), Atest@xtilde1, '--', color = 'b', linewidth = 2 , label ='Regression line')
plt.xlabel('index')
plt.ylabel('b_test')
plt.legend()
plt.show()
# -----------------------------------------------------

In [ ]:
plt.plot(np.arange(1, len(btest)+1), (Atest@xtilde1-btest), color = 'k',
         linewidth =1)
plt.xlabel('row rank')
plt.xlim(290, 600)
plt.ylabel('Error_l1norm')
plt.xticks
plt.show()

## Error estimation 

### Mean Absolute Error (MAE)
The MAE has a big advantage in that the units of the MAE are the same as the units of y, tells us the average error in units of y, the predicted feature. A value of 0 indicates a perfect fit, i.e. all our predictions are spot on. 

In [ ]:
MAE_log_normal = sum(abs(y-^y))/len(samples)
Atest@xtilde1-btest
# do this for test and for train and compare!

R squared error

### Root mean-square error
Its calculation is very similar to MAE, but instead of taking the absolute value to get rid of the sign on the individual errors, we square the error, then we can compute its sum divided by the number of samples and take its square root to get back to the same unit than y

In [ ]:
RMSE = np.sqrt(sum(np.power(y-^y, 2))/len(samples))

Compared to MAE, RMSE gives a higher total error and the gap increases as the errors become larger. It penalizes a few large errors more than a lot of small errors. If you want your model to avoid large errors, use RMSE over MAE. You should also be aware that as the sample size increases, the accumulation of slightly higher RMSEs than MAEs means that the gap between these two measures also increases as the sample size increases

### R-squared 
R2 computes how much better the regression line fits the data than the mean line. An R-squared of 1 indicates a perfect fit. An R-squared of 0 indicates a model no better or worse than the mean. An R-squared of less than 0 indicates a model worse than just predicting the mean.

In [ ]:
var_mean = sum(np.power(y - y_mean, 2))
var_line = N_samples * RMSE
R2 = (var_mean−var_line)/var_mean
## Dividing all by N: (exactly same computation)
var_mean_bis = variance(y)
var_line_bis = RMSE
R2_bis = (var_mean_bis - var_line_bis) / var_mean_bis

In [ ]:
#Using scikit learn
# MAE
mean_absolute_error(actuals, predictions)

# RMSE
sqrt(mean_squared_error(actuals, predictions)

# R-Squared
r2_score(actuals, predictions)

In [ ]:
# Regression example using sklearn
## --------------------------------------------- ##
# Define the X (input) and y (target) features
X = boston.drop("MEDV", axis=1)
y = boston["MEDV"]
# Rescale the input features
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)
# Split into train (2/3) and test (1/3) sets
test_size = 0.33
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
# Build some models (regression and k-neighbors) and check them against training data using MAE, RMSE and R2
models = [LinearRegression(), KNeighborsRegressor()]
for model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_train)
    print(type(model).__name__)
    print("    MAE", mean_absolute_error(y_train, predictions))
    print("    RMSE", sqrt(mean_squared_error(y_train, predictions)))
    print("    R2", r2_score(y_train, predictions))
    
# Evaluation the models against test data using MAE, RMSE and R2
for model in models:
    predictions = model.predict(X_test)
    print(type(model).__name__)
    print("    MAE", mean_absolute_error(y_test, predictions))
    print("    RMSE", sqrt(mean_squared_error(y_test, predictions)))
    print("    R2", r2_score(y_test, predictions))

In [ ]:
plt.plot(np.arange(1, len(btest)+1), np.power(Atest@xtilde1-btest,2), color = 'k',
         linewidth =1)
plt.xlabel('row rank')
plt.xlim(50, 120)
plt.ylabel('Error_l2norm')
plt.show()

In [ ]:
n+ np.arange(295,305)

In [ ]:
error_ind = btest.iloc[295:305].index
explored_dataset.loc[error_ind]

In [ ]:
plt.plot(np.arange(1, len(btest)+1), (Atest@xtilde1-btest))

In [ ]:
del A,b,n,p

In [ ]:
plt.plot(b, 'x', color ='k', linewidth = 2, label ='Housing values' )
plt.plot(A[sort_ind]@xtilde1, '--', color = 'b', linewidth = 1.5 , markersize=6,

In [ ]:

#nan_variables = ['LargestPropertyUseType', 'TotalGHGEmissions','GHGEmissionsIntensity'  ]
count_qualiVar = first_dataset['propertygfabuilding(s)'].value_counts()
ax = sns.barplot(x=count_qualiVar.index, y=count_qualiVar.values)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
imported_data.duplicated().sum()
imported_data.nunique()